In [ ]:
import pandas as pd
import numpy as np
from numpy.core.umath_tests import inner1d
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from Utilities.utils import *
from Utilities.plot_functions import *

# Set aSBP as the prediction variable
prediction_variable = 'aSBP'  

# Select regression method, e.g., 'RF','SVR','RIDGE','GB'
regressor = 'SVR'       

# Enable figure saving
save_figure_boolean = False 

experiment_type = 'invivo'    # Set the type of data being used in the analysis

# Load Data

In [ ]:
invivo_dataset = pd.read_csv('Data/invivo_data.csv')
insilico_dataset = pd.read_csv('Data/insilico_data.csv')
# insilico_dataset.head()

# Regression Analysis

## Validation of an in vivo trained SVR using in vivo data

In [ ]:
# - Seperate attributes and prediction values
X = invivo_dataset.iloc[:,invivo_dataset.columns.get_loc('brSBP'):-1].values
y = invivo_dataset.iloc[:,-1].values
# save_input_output_pairs(X,y)

# X,y = load_input_output_pairs()

In [ ]:
# k-fold Cross-validation
k = 10
kf = KFold(n_splits=k); kf.get_n_splits(X); print(kf); KFold(n_splits=k, random_state=None, shuffle=False)

current_fold = 0;
y_pred_total = None; y_test_total = None;
metrics_array = np.empty((k, 4))


for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
        
    # model, y_pred, hyper_parameters = hyperparameter_tuning(X_train, X_test, y_train, y_test,regressor)
    model, y_pred = run_regressor_with_best_hyperparams(X_train, X_test, y_train, y_test, regressor, prediction_variable)
    
    output_verbose = False
    metrics_array = calculate_metrics_for_each_fold(current_fold,y_test,y_pred,metrics_array,output_verbose)
    current_fold = current_fold + 1
    
    y_test_total, y_pred_total = accumulate_predictions(y_test, y_pred, current_fold, y_test_total, y_pred_total)
    
    

### Print Results

In [ ]:
calculate_regression_statistics(y_test_total, y_pred_total, metrics_array, regressor)

### Plot Results

In [ ]:
plot_data(y_test_total, y_pred_total, experiment_type, prediction_variable, regressor, save_figure_boolean)

## Validation of an in silico trained SVR using in vivo data

In [ ]:
X_train = insilico_dataset[['brSBP','brDBP','cfPWV','HR']].values
y_train = insilico_dataset[['aSBP']].values

X_test = invivo_dataset[['brSBP','brDBP','cfPWV','HR']].values
y_test = invivo_dataset[['aSBP']].values.flatten()

metrics_array = np.empty((1, 4))
model, y_pred = run_regressor_with_best_hyperparams(X_train, X_test, y_train, y_test, regressor, prediction_variable)
output_verbose = False
metrics_array = calculate_metrics_for_each_fold(0,y_test, y_pred,metrics_array,output_verbose)

### Print Results

In [ ]:
calculate_regression_statistics(y_test, y_pred, metrics_array, regressor)

### Plots Results

In [ ]:
plot_data(y_test, y_pred, experiment_type, prediction_variable, regressor, save_figure_boolean)